In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

ID: 1485ba305ac1f000

In [4]:
#importing libraries
import pandas as pd
import numpy as np
from os import listdir
from numpy import zeros
from numpy import asarray
from numpy import savez_compressed
from pandas import read_csv
from keras import backend
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import os
import time
from sklearn.metrics import fbeta_score
import tensorflow as tf
import sys
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model


In [5]:
# Libraries for Data Visualisation
import matplotlib.pyplot as plt
from matplotlib import pyplot
from matplotlib.image import imread
%matplotlib inline

In [6]:
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split

In [7]:
#import train_classes.csv, test and train jpg folder
train_classes = pd.read_csv('../input/planet/planet/planet/train_classes.csv')


In [9]:

test_jpg_datasets = '../input/planet/planet/planet/test-jpg/'
train_jpg_datasets = '../input/planet/planet/planet/train-jpg/'

In [10]:
#view train datasets
train_classes.head()

In [12]:
plt.figure(figsize=(15,15))
folder =  "planet/train-jpg/"
for i in range(12):
    pyplot.subplot(330 + 1 + i)

    file = train_jpg_datasets + 'train_' + str(i) + '.jpg'
    
    #Load pixels of the images
    images = imread(file)
    pyplot.imshow(images)
    
plt.show()

In [14]:
lists = []
for tag in train_classes.tags.values:
    dataset = tag.split(' ')
    for data in dataset:
        if data not in lists:
            lists.append(data)
            
len(lists)

In [16]:
mapping = {lists[i]: i for i in range(len(lists))}
invert_mapping= {i:lists[i] for i in range(len(lists))}
invert_mapping

In [18]:
# Map files to tags
def file_mapper(data_csv):
    mapper = dict()
    for i in range(len(data_csv)):
        name, tags = data_csv['image_name'][i], data_csv['tags'][i]
        mapper[name] = tags.split(' ')
    return mapper
mapped_file = file_mapper(train_classes)

In [20]:
#function to categorize images using one_hot_encoder
def one_hot_encoder(tags, mapper):
    
    #Creating an empty matrix(Vector)
    encoding = np.zeros(len(mapper), dtype='uint8')
    
    #Map 1 for each tag in the vector created
    for t in tags:
        encoding[mapper[t]] = 1
    return encoding

In [26]:
def load_dataset(path, file_mapping, mapping):
    photos = list()
    targets = list()
    
    for file in os.listdir(train_jpg_datasets):
        
        #Load Images
        photo = load_img(path + file, target_size = (64, 64))
        
        #Converting photos to numpy array
        photo = img_to_array(photo, dtype = 'uint8')
        
        #Tags
        tags = mapped_file[file[:-4]]
        
        target = one_hot_encoder(tags, tag_mapping)
        
        #Stores
        photos.append(photo)
        targets.append(target)
    X = np.asarray(photos,dtype='uint8')
    Y = np.asarray(targets,dtype='uint8')
    return X,Y

X, Y = load_dataset(train_jpg_datasets, mapped_file, mapping)
print(X.shape, Y.shape)

# Compressing X and Y arrays into one single file
np.savez_compressed('testtraindata.zip', X, Y)